In [1]:
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU
import numpy as np

In [2]:
#Parameters
K_bar = 1
num_generated_channel = 1
batch_size = 32
skip_connections = True


In [4]:
#ENCODER
encoder_input = tf.keras.Input(shape=(96,84,1))
encoder_pitchconv1 = tf.keras.layers.Conv2D(16,(1,12),(1,12),activation=LeakyReLU())(encoder_input)
encoder_bn1 = tf.keras.layers.BatchNormalization()(encoder_pitchconv1)
encoder_pitchconv2 = tf.keras.layers.Conv2D(16,(1,7),(1,7),activation=LeakyReLU())(encoder_bn1)
encoder_bn2 = tf.keras.layers.BatchNormalization()(encoder_pitchconv2)
encoder_timeconv1 = tf.keras.layers.Conv2D(16,(3,1),(3,1),activation=LeakyReLU())(encoder_bn2)
encoder_bn3 = tf.keras.layers.BatchNormalization()(encoder_timeconv1)
encoder_timeconv2 = tf.keras.layers.Conv2D(16,(2,1),(2,1),activation=LeakyReLU())(encoder_bn3)
encoder_bn4 = tf.keras.layers.BatchNormalization()(encoder_timeconv2)
encoder_timeconv3 = tf.keras.layers.Conv2D(16,(2,1),(2,1),activation=LeakyReLU())(encoder_bn4)
encoder_bn5 = tf.keras.layers.BatchNormalization()(encoder_timeconv3)
encoder_timeconv4 = tf.keras.layers.Conv2D(16,(2,1),(2,1),activation=LeakyReLU())(encoder_bn5)
encoder_bn6 = tf.keras.layers.BatchNormalization()(encoder_timeconv4)
encoder_output = tf.keras.layers.Flatten()(encoder_bn6)

decoder_input = tf.keras.layers.Reshape((4,1,16))(encoder_output)
decoder_timeconv1 = tf.keras.layers.Conv2DTranspose(16,(2,1),(2,1),activation=LeakyReLU())(decoder_input)
decoder_bn1 = tf.keras.layers.BatchNormalization()(decoder_timeconv1)
decoder_out1 = tf.keras.layers.add([decoder_bn1,encoder_bn5])
decoder_timeconv2 = tf.keras.layers.Conv2DTranspose(16,(2,1),(2,1),activation=LeakyReLU())(decoder_out1)
decoder_bn2 = tf.keras.layers.BatchNormalization()(decoder_timeconv2)
decoder_out2 = tf.keras.layers.add([decoder_bn2,encoder_bn4])
decoder_timeconv3 = tf.keras.layers.Conv2DTranspose(16,(2,1),(2,1),activation=LeakyReLU())(decoder_out2)
decoder_bn3 = tf.keras.layers.BatchNormalization()(decoder_timeconv3)
decoder_out3 = tf.keras.layers.add([decoder_bn3,encoder_bn3])
decoder_timeconv4 = tf.keras.layers.Conv2DTranspose(16,(3,1),(3,1),activation=LeakyReLU())(decoder_out3)
decoder_bn4 = tf.keras.layers.BatchNormalization()(decoder_timeconv4)
decoder_out4 = tf.keras.layers.add([decoder_bn4,encoder_bn2])
decoder_pitchconv1 = tf.keras.layers.Conv2DTranspose(16,(1,7),(1,7),activation=LeakyReLU())(decoder_out4)
decoder_bn5 = tf.keras.layers.BatchNormalization()(decoder_pitchconv1)
decoder_out5 = tf.keras.layers.add([decoder_bn5,encoder_bn1])
decoder_pitchconv2 = tf.keras.layers.Conv2DTranspose(1,(1,12),(1,12),activation='sigmoid')(decoder_out5)
decoder_output = tf.keras.layers.BatchNormalization()(decoder_pitchconv2)


autoencoder= tf.keras.Model(encoder_input,decoder_output,name="autoencoder")
autoencoder.summary()

Model: "autoencoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 96, 84, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 96, 7, 16)    208         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 96, 7, 16)    64          conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 96, 1, 16)    1808        batch_normalization_12[0][0]     
________________________________________________________________________________________

In [5]:
encoder = tf.keras.Model(encoder_input,encoder_output,name="encoder")
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 84, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 96, 7, 16)         208       
_________________________________________________________________
batch_normalization_12 (Batc (None, 96, 7, 16)         64        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 96, 1, 16)         1808      
_________________________________________________________________
batch_normalization_13 (Batc (None, 96, 1, 16)         64        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 1, 16)         784       
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 1, 16)         64  

In [ ]:
#we update G once every five updates of D and apply batch normalization only to G

In [6]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [49]:
#For the Autoencoder, train one version on piano and one version on orchestra

(32, 64)

In [ ]:
#1. implement skip connections
#2. build also the generateor and discriminator based on pix2pix version as well